In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfTransformer
import sys
sys.path.append("../WSM2020_final_dmmig/utils/")
from recsys_metric import mrr
# print(sys.path, sys.modules)

time: 327 ms (started: 2022-06-01 15:45:43 +00:00)


# Load data

## test data

In [4]:
test_data = "leaderboard"

if test_data == "leaderboard":
    test_session = "test_leader_session.pkl"
elif test_data == "final":
    test_session = "test_session.pkl"


with open(f"./datasets/{test_session}" , "rb") as f:
    test_dic = pickle.load(f)
    
test_df = pd.DataFrame.from_dict(test_dic, orient='index')

del test_dic

time: 6.56 s (started: 2022-06-01 15:45:58 +00:00)


In [5]:
display(f"test_df: {test_df.shape}", test_df.head())
display(test_df.iloc[1,(test_df.iloc[1,:]>0).tolist()], "-"*100)

'test_df: (50000, 904)'

,0,1,2,3,4,5,6,7,8,9,...,894,895,896,897,898,899,900,901,902,903
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


146    4.0
176    1.0
178    3.0
183    4.0
219    2.0
242    2.0
271    1.0
290    4.0
401    4.0
440    4.0
449    1.0
452    2.0
470    1.0
489    1.0
490    3.0
507    4.0
571    4.0
663    4.0
669    4.0
692    4.0
733    1.0
743    3.0
807    4.0
855    4.0
859    4.0
873    4.0
Name: 200, dtype: float64

'----------------------------------------------------------------------------------------------------'

time: 20.1 ms (started: 2022-06-01 15:46:05 +00:00)


## train data & candidate data

In [48]:
data_period = "2-week"

if data_period == "1-month":
    train_session = "train_session_last_1m.pkl"
    purchases_session = "purchases_session_last_1m.pkl"
    train_purchases_session = "train_purchases_session_last_1m.pkl"
    candidate_vector = "candidate_last_1m_vector"
    train_Y_date = "2021-5-1"
elif data_period == "3-month":
    train_session = "train_session_last_3m.pkl"
    purchases_session = "purchases_session_last_3m.pkl"
    train_purchases_session = "train_purchases_session_last_3m.pkl"
    candidate_vector = "candidate_last_3m_vector"
    train_Y_date = "2021-3-1"
elif data_period == "2-week":
    train_session = "train_session_last_05m.pkl"
    purchases_session = "purchases_session_last_05m.pkl"
    train_purchases_session = "train_purchases_session_last_05m.pkl"
    candidate_vector = "candidate_last_05m_vector"
    train_Y_date = "2021-5-15"
elif data_period == "all-time":
    train_session = "train_session.pkl"
    purchases_session = "purchases_session.pkl"
    train_purchases_session = "train_purchases_session.pkl"
    candidate_vector = "candidate_vector"
    train_Y_date = "2020-1-1"
    
with open(f"./datasets/{train_session}" , "rb") as f:
    train_dic = pickle.load(f)
    
with open(f"./datasets/{purchases_session}" , "rb") as f:
    purchases_dic = pickle.load(f)
    
with open(f"./datasets/{train_purchases_session}" , "rb") as f:
    train_purchases_dic = pickle.load(f)
    

train_df= pd.DataFrame.from_dict(train_dic, orient='index')
purchases_df= pd.DataFrame.from_dict(purchases_dic, orient='index')
train_purchases_df= pd.DataFrame.from_dict(train_purchases_dic, orient='index')
del train_dic, purchases_dic, train_purchases_dic


train_Y_raw = pd.read_csv("../recsys2022/train_purchases.csv")
train_Y_raw['date'] = pd.to_datetime(train_Y_raw['date'])
train_Y = train_Y_raw.iloc[(train_Y_raw['date'] >= f'{train_Y_date}').tolist(), :]


candidate_df = pd.read_csv(f"./datasets/{candidate_vector}.csv").set_index("item_id")
candidate_ri_df = candidate_df.reset_index()

time: 18.5 s (started: 2022-06-01 16:55:10 +00:00)


In [49]:
display(f"train_df: {train_df.shape}", train_df.head(), "-"*100)
display(f"purchases_df: {purchases_df.shape}", purchases_df.head(), "-"*100)
display(f"train_purchases_df: {train_purchases_df.shape}", train_purchases_df.head(), "-"*100)
display(f"train_Y: {train_Y.shape}", train_Y, "-"*100)
display(f"candidate: {candidate_df.shape}", candidate_df.head(), "-"*100)
display(f"candidate reset index: {candidate_ri_df.shape}", candidate_ri_df.head(), "-"*100)

'train_df: (47320, 904)'

,0,1,2,3,4,5,6,7,8,9,...,894,895,896,897,898,899,900,901,902,903
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


'----------------------------------------------------------------------------------------------------'

'purchases_df: (47320, 904)'

,0,1,2,3,4,5,6,7,8,9,...,894,895,896,897,898,899,900,901,902,903
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


...

'----------------------------------------------------------------------------------------------------'

'candidate reset index: (3742, 905)'

,item_id,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,...,7_837,7_838,8_157,8_323,8_420,8_636,9_175,9_178,9_190,9_369
0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


'----------------------------------------------------------------------------------------------------'

time: 68.2 ms (started: 2022-06-01 16:55:29 +00:00)


## Concat raw data 

In [51]:
train_con_purchasees_df = pd.concat([train_df, purchases_df])
train_con_test_df = pd.concat([train_df, test_df])
purchases_con_test_df = pd.concat([purchases_df, test_df])
train_purchases_con_test_df = pd.concat([train_purchases_df, test_df])

time: 338 ms (started: 2022-06-01 17:03:58 +00:00)


# Build tfidf vector

- fit tfidf model
- turn `candidte_df` to tfidf vector

In [63]:
tfidf_train_data = train_purchases_con_test_df
# tfidf_train_data = train_con_test_df
# tfidf_train_data = train_purchases_df
# tfidf_train_data = train_df
# tfidf_train_data = purchases_df
# tfidf_train_data = train_con_purchasees_df

tfidf_model = TfidfTransformer()
tfidf_model.fit(tfidf_train_data)
candidate_tfidf_sparse_mat = tfidf_model.transform(candidate_df)
candidate_tfidf_arr = candidate_tfidf_sparse_mat.toarray()
del candidate_tfidf_sparse_mat

time: 593 ms (started: 2022-06-02 15:52:08 +00:00)


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:438: UserWarning: X has feature names, but TfidfTransformer was fitted without feature names
  warnings.warn(


In [53]:
print(f"candidate_tfidf_arr shape:{candidate_tfidf_arr.shape}")
display(candidate_tfidf_arr[0])

candidate_tfidf_arr shape:(3742, 904)


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

time: 5.42 ms (started: 2022-06-01 17:04:00 +00:00)


# Compute similarity 
- similarity bewteen session_tfidf and candidate_tfidf
- rank by similarity

In [54]:
def rank_tfidf_sim(target_df, model, overview=True):
    session_tfidf_sparse_mat = model.transform(target_df)
    session_tfidf_arr = session_tfidf_sparse_mat.toarray()
    del session_tfidf_sparse_mat
    sim_arr = np.dot(session_tfidf_arr,candidate_tfidf_arr.T) # similarity bewteen session_tfidf and candidate_tfidf
    argsort_sim_arr = np.argsort(sim_arr)[::,::-1] # rank by similarity
    item_id_rank_arr = np.apply_along_axis(lambda x:candidate_ri_df.loc[x[:100],"item_id"], 1, argsort_sim_arr).reshape(-1)
    res_df = pd.DataFrame(np.repeat(target_df.index, 100), columns=["session_id"])
    res_df["item_id"] = item_id_rank_arr
    res_df["rank"] = (list(range(1,101))*len(target_df))
    
    if (overview):
        display(f"session_tfidf_arr shape:{session_tfidf_arr.shape}",session_tfidf_arr[0], "-"*100)
        display(f"rank by similarity:", sim_arr[0][argsort_sim_arr[0]], "-"*100)
        display(f"argsort of sim_arr:", argsort_sim_arr, "-"*100)
        display(f"item_id_rank_arr[:10]:", item_id_rank_arr[:10], "-"*100)
        observe_ind = np.array([[i*100+j for j in range(10)] for i in range(len(target_df))]).reshape(-1)
        display(f"overview rank df:", res_df.iloc[observe_ind,:][:50], "-"*100)
        
    return res_df

time: 889 µs (started: 2022-06-01 17:04:04 +00:00)


# evaluation

In [64]:
eva_data = train_df
eva_label = train_Y

eva_res = rank_tfidf_sim(eva_data, tfidf_model, overview=False)
mrr_score = mrr(eva_res, eva_label, overview=False)

print(mrr_score)
display(f"eva_res shape:{eva_res.shape}", eva_res.iloc[95:105,:])

0.058460915276370984


'eva_res shape:(4732000, 3)'

,session_id,item_id,rank
95,115,2759,96
96,115,12508,97
97,115,10874,98
98,115,15081,99
99,115,24574,100
100,261,17428,1
101,261,3946,2
102,261,22387,3
103,261,6625,4
104,261,14216,5


time: 19.1 s (started: 2022-06-02 15:52:12 +00:00)


# Output results

In [56]:
res = rank_tfidf_sim(test_df, tfidf_model)
res.to_csv("./datasets/results/leader_tfidf_train_purchases_con_test_last_05_0601.csv", index=False)
del res

'session_tfidf_arr shape:(50000, 904)'

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.32175736, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

'----------------------------------------------------------------------------------------------------'

'rank by similarity:'

array([1.        , 0.95425342, 0.91871144, ..., 0.00757757, 0.0070193 ,
       0.00685818])

...

'----------------------------------------------------------------------------------------------------'

'overview rank df:'

,session_id,item_id,rank
0,26,19185,1
1,26,3260,2
2,26,17740,3
3,26,22166,4
4,26,18418,5
5,26,7109,6
6,26,22736,7
7,26,5199,8
8,26,23612,9
9,26,16604,10


'----------------------------------------------------------------------------------------------------'

time: 23.4 s (started: 2022-06-01 17:05:26 +00:00)
